In [30]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')
nltk.download('vader_lexicon')
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to C:\Users\Husein
[nltk_data]     Ghadiali\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Husein Ghadiali\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to C:\Users\Husein
[nltk_data]     Ghadiali\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Husein
[nltk_data]     Ghadiali\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [60]:
df = pd.read_csv('complaints.csv')

In [62]:
#Checking for null values
df.isnull().sum()

Date received                         0
Product                               0
Sub-product                      235291
Issue                                 0
Sub-issue                        710080
Consumer complaint narrative    2382452
Company public response         2037159
Company                               0
State                             41560
ZIP code                          30192
Tags                            3330738
Consumer consent provided?       876716
Submitted via                         0
Date sent to company                  0
Company response to consumer          5
Timely response?                      0
Consumer disputed?              2964523
Complaint ID                          0
dtype: int64

In [63]:
# Dropping unnecessary columns
df.drop(['Date received', 'Product', 'Sub-product','Sub-issue','Company public response', 'Company', 'State', 'ZIP code', 'Tags', 'Consumer consent provided?', 'Submitted via', 'Date sent to company', 'Company response to consumer', 'Timely response?', 'Consumer disputed?', 'Complaint ID'], axis=1, inplace=True)

In [64]:
df.isna().sum()

Issue                                 0
Consumer complaint narrative    2382452
dtype: int64

In [65]:
# Dropping null values
df.dropna(inplace=True)

In [68]:
# Renaming columns
df.rename(columns={'Issue':'issue', 'Consumer complaint narrative':'complaint'}, inplace=True)
df.head()


,issue,complaint
0,Problem with a credit reporting company's inve...,A few weeks ago I sent letters to all 3 credit...
3,Problem with a credit reporting company's inve...,I have submitted several disputes with the fol...
4,Improper use of your report,In accordance with the Fair Credit Reporting a...
5,Problem with a credit reporting company's inve...,All XXXX XXXX XXXX XXXX account are not meetin...
6,Problem with a credit reporting company's inve...,"Despite multiple written requests, the unverif..."


In [69]:
# Map treebank part of speech tags to WordNet part of speech tags
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [70]:
# Preprocess text by removing punctuation, stop words, and lemmatizing tokens
def preprocess_text(text):
    # Remove standalone occurrences of 'X' or 'x'
    text = ' '.join([word for word in text.split() if not all(c in 'Xx' for c in word)])

    # Remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(text)
    tokens = [token for token in tokens if token.lower() not in stop_words]
    
    # Lemmatize tokens using their part of speech tags
    lemmatizer = WordNetLemmatizer()
    pos_tags = nltk.pos_tag(tokens)
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(pos)) for token, pos in pos_tags]
    
    return ' '.join(lemmatized_tokens)

In [71]:
# Apply the preprocess_text function to the Review column
df['complaint'] = df['complaint'].apply(preprocess_text)

In [72]:
df.head()

,issue,complaint
0,Problem with a credit reporting company's inve...,week ago send letter 3 credit bureau regard it...
3,Problem with a credit reporting company's inve...,submit several dispute follow credit report ag...
4,Improper use of your report,accordance Fair Credit Reporting act List acco...
5,Problem with a credit reporting company's inve...,account meeting FCRA requirement multiple viol...
6,Problem with a credit reporting company's inve...,Despite multiple write request unverified item...


In [77]:
df.shape

(1350429, 2)

In [78]:
# Save to CSV file
df.to_csv('complaintsclean.csv', index=False)